# Amazon Bedrock AgentCore 런타임(Runtime)에서 OpenAI 모델을 사용한 Strands 에이전트(Agent) 호스팅

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore 런타임(Runtime)을 사용하여 기존 에이전트(Agent)를 호스팅하는 방법을 배웁니다.

OpenAI 모델을 사용한 Strands 에이전트(Agent) 예제에 초점을 맞출 것입니다. Amazon Bedrock 모델을 사용한 Strands 에이전트(Agent)는 [여기](../01-strands-with-bedrock-model)를, Amazon Bedrock 모델을 사용한 LangGraph는 [여기](../02-langgraph-with-bedrock-model)를 확인하세요.


### 튜토리얼 세부사항

| 정보               | 세부사항                                                                 |
|:--------------------|:------------------------------------------------------------------------|
| 튜토리얼 유형       | 대화형                                                                  |
| 에이전트(Agent) 유형 | 단일                                                                    |
| 에이전틱 프레임워크(Agentic Framework) | Strands Agents                                        |
| LLM 모델            | GPT 4.1 mini                                                            |
| 튜토리얼 구성 요소   | AgentCore 런타임(Runtime)에 에이전트(Agent) 호스팅. Strands Agent와 OpenAI 모델 사용 |
| 튜토리얼 분야       | 범용                                                                    |
| 예제 복잡도         | 쉬움                                                                    |
| 사용 SDK            | Amazon BedrockAgentCore Python SDK 및 boto3                             |

### 튜토리얼 아키텍처

이 튜토리얼에서는 기존 에이전트(Agent)를 AgentCore 런타임(Runtime)에 배포(Deployment)하는 방법을 설명합니다.

데모 목적으로 Amazon Bedrock 모델을 사용하는 Strands 에이전트(Agent)를 사용할 것입니다.

예제에서는 `get_weather`와 `get_time` 두 가지 도구(Tool)를 가진 매우 간단한 에이전트(Agent)를 사용합니다.

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### 튜토리얼 주요 기능

* Amazon Bedrock AgentCore 런타임(Runtime)에 에이전트(Agent) 호스팅
* OpenAI 모델 사용
* Strands 에이전트(Agent) 사용

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker 실행 중

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [ ]:
%%writefile strands_agents_openai.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from strands.models.litellm import LiteLLMModel
import os

os.environ["AZURE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

model = "azure/gpt-4.1-mini"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
)


agent = Agent(
    model=litellm_model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_open_ai(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_open_ai(json.loads(args.payload))
    print(response)

## 에이전트(Agent) 생성 및 로컬에서 실험하기

에이전트(Agent)를 AgentCore 런타임(Runtime)에 배포(Deployment)하기 전에 실험 목적으로 로컬에서 개발하고 실행해 보겠습니다.

프로덕션 에이전틱 애플리케이션의 경우 에이전트(Agent) 생성 프로세스와 에이전트(Agent) 호출(Invocation) 프로세스를 분리해야 합니다. AgentCore 런타임(Runtime)에서는 에이전트(Agent)의 호출(Invocation) 부분을 `@app.entrypoint` 데코레이터로 장식하고 이를 런타임(Runtime)의 진입점으로 사용합니다. 먼저 실험 단계에서 각 에이전트(Agent)가 어떻게 개발되는지 살펴보겠습니다.

여기서 아키텍처는 다음과 같습니다:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [ ]:
!python strands_agents_openai.py '{"prompt": "What is the weather now?"}'

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agents with OpenAI model
Let's start with our Strands Agent using the GPT 4.1 mini model. All the others will work exactly the same.

In [ ]:
%%writefile strands_agents_openai.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from strands.models.litellm import LiteLLMModel
import os
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

os.environ["AZURE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

model = "azure/gpt-4.1-mini"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
)


agent = Agent(
    model=litellm_model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)


@app.entrypoint
def strands_agent_open_ai(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

#### 로컬 에이전트(Agent) 호출(Invocation)

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

## AgentCore 런타임(Runtime) 배포(Deployment)를 위한 에이전트(Agent) 준비

이제 에이전트(Agent)를 AgentCore 런타임(Runtime)에 배포(Deployment)해 보겠습니다. 이를 위해 다음이 필요합니다:
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp`으로 Runtime App 임포트
* 코드에서 `app = BedrockAgentCoreApp()`으로 App 초기화
* 호출(Invocation) 함수를 `@app.entrypoint` 데코레이터로 장식
* `app.run()`으로 AgentCoreRuntime이 에이전트(Agent) 실행을 제어하도록 함

### OpenAI 모델을 사용한 Strands 에이전트(Agent)
GPT 4.1 mini 모델을 사용하는 Strands 에이전트(Agent)부터 시작해 보겠습니다. 다른 모든 것도 정확히 동일하게 작동합니다.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import boto3
import json
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

agent_name = "strands_openai_getting_started"
response = agentcore_runtime.configure(
    entrypoint="strands_agents_openai.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

## 내부적으로 무슨 일이 일어나나요?

`BedrockAgentCoreApp`을 사용하면 자동으로:

* 포트 8080에서 수신 대기하는 HTTP 서버 생성
* 에이전트(Agent) 요구사항 처리를 위한 필수 `/invocations` 엔드포인트(Endpoint) 구현
* 헬스 체크를 위한 `/ping` 엔드포인트(Endpoint) 구현 (비동기 에이전트(Agent)에 매우 중요)
* 적절한 콘텐츠 타입 및 응답 형식 처리
* AWS 표준에 따른 오류 처리 관리

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

## AgentCore 런타임(Runtime)에 에이전트(Agent) 배포(Deployment)

`CreateAgentRuntime` 작업은 포괄적인 구성(Configuration) 옵션을 지원하여 컨테이너 이미지, 환경 변수 및 암호화 설정을 지정할 수 있습니다. 또한 프로토콜 설정(HTTP, MCP)과 인가(Authorization) 메커니즘을 구성(Configuration)하여 클라이언트가 에이전트(Agent)와 통신하는 방식을 제어할 수 있습니다.

**참고:** 운영 모범 사례는 코드를 컨테이너로 패키징하고 CI/CD 파이프라인 및 IaC를 사용하여 ECR에 푸시하는 것입니다.

이 튜토리얼에서는 Amazon Bedrock AgentCode Python SDK를 사용하여 아티팩트를 쉽게 패키징하고 AgentCore 런타임(Runtime)에 배포(Deployment)할 수 있습니다.

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "Hi, what can you do?"})
invoke_response

### AgentCore 런타임(Runtime) 배포(Deployment) 구성(Configuration)

다음으로 스타터 툴킷을 사용하여 진입점, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore 런타임(Runtime) 배포(Deployment)를 구성(Configuration)합니다. 또한 시작 시 Amazon ECR 리포지토리를 자동 생성하도록 스타터 키트를 구성(Configuration)합니다.

구성(Configuration) 단계에서 애플리케이션 코드를 기반으로 도커 파일이 생성됩니다.

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2X2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

### AgentCore 런타임(Runtime)에 에이전트(Agent) 시작

이제 도커 파일이 준비되었으니 에이전트(Agent)를 AgentCore 런타임(Runtime)에 시작해 보겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 AgentCore 런타임(Runtime)이 생성됩니다.

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!